In [1]:
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('src/')
sys.dont_write_bytecode = True
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import transformers
import torch
import transformers 
import torch
from util import unshard_deepspeed, generate_text

# Llama 7b memory footprint with LORA r = 16

# When create - use a large disk for the checkpoints

# Inference 16bit - 13gb, 8bit - 8gb

# Train 2batch - 16bit -  > 49 gb
# Train 2batch - deepspeed_stage_2 - 16bit -  27gb
# Train 4batch - deepspeed_stage_2 - 16bit -  38gb
# Train 4batch - deepspeed_stage_2 - 16bit -  43gb
# Train 4batch - deepspeed_stage_3 - 16bit -  45gb
# Train 6batch - deepspeed_stage_2 - 16bit -  > 49gb

In [2]:
from reacher.reacher import Reacher, ReacherDocker, RemoteClient
from reacher.reacher import create_notebook, create_tensorboard

reacher = Reacher(
    build_name="training_alpacha_lora",
    host="",
    port=8961,
    user="root",
    ssh_key_filepath=".ssh/id_rsa",
    prefix_cmd="PATH=/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home//.local/bin"
)

reacher.setup()

In [3]:
reacher.execute_command("nvidia-smi")

Sun Apr 16 08:29:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:1B:00.0 Off |                  Off |
| 30%   32C    P8    19W / 300W |      0MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:B2:00.0 Off |                  Off |
| 30%   

In [ ]:
# Create a notebook session on the remote and do port-forwarding between the remote and your local 
create_notebook(reacher, remote_port=55068, local_port=55068)

In [18]:
# Create a tensorboard session on the remote and do port-forwarding between the remote and your local 
create_tensorboard(reacher, remote_port=55069, local_port=55069, logdir=reacher.build_path+"/artifacts/runs")

tensorboard running on
http://0.0.0.0:55069/


In [13]:
reacher.list_named_sessions()


	7704.tensorboard	(04/16/23 08:46:05)	(Attached)



In [ ]:
reacher.kill_named_session("tensorboard")

## Training

### Send all data to the remote

In [ ]:
reacher.put(["src", "data", "config.yaml"])

In [ ]:
# Process the data
reacher.execute(
    "python src/setup.py --local --model 'decapoda-research/llama-7b-hf' --data_file alpaca_data_cleaned.json --max_tokens 512",
    context=["src", "data/alpaca_data_cleaned.json"],
    wrap_in_screen=True,
    named_session="setup",
)

In [ ]:
# Train the model
reacher.execute(
    "python src/train_lightning.py --config config.yaml --local",
    context=["src", "config.yaml"],
    wrap_in_screen=True,
    named_session="training",
)

In [17]:
reacher.execute_command("nvidia-smi")

Sun Apr 16 10:05:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:1B:00.0 Off |                  Off |
| 59%   83C    P2   284W / 300W |  46816MiB / 49140MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:B2:00.0 Off |                  Off |
| 47%   

In [ ]:
reacher.kill_named_session("training")

In [ ]:
# Grab the checkpoint
reacher.get("artifacts...../final.ckpt", "weights/")